# Utilize Word2Vec & FastText Model

In [1]:
import sys
import gc
import os
import re
import time
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
from tqdm import tqdm_notebook
# import chromedriver_autoinstaller

from matplotlib import font_manager, rc
# 차트에서 한글 출력을 위한 설정

import matplotlib.pyplot as plt
import platform
your_os = platform.system()
if your_os == 'Linux':
    rc('font', family='NanumGothic')
elif your_os == 'Windows':
    ttf = "c:/Windows/Fonts/malgun.ttf"
    font_name = font_manager.FontProperties(fname=ttf).get_name()
    
    rc('font', family=font_name)
elif your_os == 'Darwin':
    rc('font', family='AppleGothic')
rc('axes', unicode_minus=False)

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common. by import By

from bs4 import BeautifulSoup 

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

import konlpy
from konlpy.tag import Okt
from konlp.kma.klt2023 import klt2023
import nltk
import stanza
from gensim.models import Word2Vec, word2vec
from gensim.models import FastText

from wordcloud import WordCloud
from numpy import dot
from numpy.linalg import norm
import itertools

#### 코사인 유사도 함수 정의

In [3]:
# 코사인 유사도 함수

def cosine_similarity(v1, v2):
    dot_product = np.dot(v1, v2)
    norm_v1 = np.linalg.norm(v1)
    norm_v2 = np.linalg.norm(v2)
    return (dot_product / (norm_v1 * norm_v2)).astype('float')

## 도서 내용으로  도서 추천
- 문장을 입력하면 해당 문장과 도서 내용의 유사도를 계산하여 유사도가 높은 도서를 추천해주는 함수

In [4]:
def book_recommend(model_name, dataframe, input_sentence, n):
    
    # 모델 호출
    try:
        model = Word2Vec.load(model_name)
    except:
        model = FastText.load(model_name)
    
    # 데이터프레임 호출
    book = pd.read_csv(dataframe)
    
    k = klt2023()
    
    input_nouns = list(set(k.nouns(input_sentence)))
    
    # 단어의 벡터 합 계산
    iv=np.zeros(400)
    input_vec =[]
    for k in input_nouns:
        iv += model.wv.get_vector(k)
    
    # 책벡터와 유사도 계산
    cos_num = []
    for i in range(len(book)):
        book_vector = [np.float(i) for i in book['책소개_벡터'][i][1:-1].split()]
        nn = cosine_similarity(iv, book_vector)
        cos_num.append(nn.astype('float'))
    
    if len(cos_num) == len(book):
        cos_dic = dict(zip(zip(book['제목'],book['저자']), cos_num))
        similar_sentence = sorted(cos_dic.items(), key=lambda x: x[1], reverse = True)
        return similar_sentence[:n]
    else:
        return len(cos_num)

In [8]:
book_recommend('word2vec_book_wiki.model', 'books_vector_wiki.csv', '마음을 편안하게 해주는 위로의 글', 5)

[(('행복을 담아줄게', '나란'), 0.35825759059687917),
 (('있는 그대로', '김지훈'), 0.3403578870621218),
 (('보이지 않는 곳에서 애쓰고 있는 너에게', '최대호'), 0.33851055987823103),
 (('당신과 아침에 싸우면 밤에는 입맞출 겁니다', '유래혁'), 0.3367892482086697),
 (('엄마의 말 공부 일력 365', '이임숙'), 0.3345570815563324)]

In [9]:
book_recommend('fasttext_book_wiki.model', 'books_vector_wiki.csv', '마음을 편안하게 해주는 위로의 글', 5)

[(('당신이 좋아지면, 밤이 깊어지면', '안희연'), 0.7639079225989486),
 (('나로서 충분히 괜찮은 사람', '김재식'), 0.7610791092550505),
 (('당신과 아침에 싸우면 밤에는 입맞출 겁니다', '유래혁'), 0.7595994689581971),
 (('나는 당신이 행복했으면 좋겠습니다', '박찬위'), 0.7592670581220978),
 (('그대 늙어가는 것이 아니라 익어가는 것이다', '오평선'), 0.7533357641240881)]

In [40]:
book_recommend('word2vec_book_wiki.model', 'books_vector_wiki.csv', '부자로 성공하는 방법', 5)

[(('백만장자 시크릿', '하브 에커'), 0.399624876268301),
 (('멘탈의 연금술', '보도 섀퍼'), 0.36896506015454933),
 (('10배의 법칙', '그랜트 카돈'), 0.3657964732238805),
 (('부의 본능(골드 에디션)', '우석'), 0.36520110889494883),
 (('Give and Take(기브앤테이크)', '애덤 그랜트'), 0.36266554809804324)]

In [41]:
book_recommend('fasttext_book_wiki.model', 'books_vector_wiki.csv', '부자로 성공하는 방법', 5)

[(('경매 권리분석 이렇게 쉬웠어?', '박희철(파이팅팔콘)'), 0.64510457835586),
 (('장단기 투자의 비밀', '래리 윌리엄스'), 0.6328833359318223),
 (('백만장자 시크릿', '하브 에커'), 0.6326274139210921),
 (('10대를 위한 그릿', '매슈사이드'), 0.627665727245654),
 (('Give and Take(기브앤테이크)', '애덤 그랜트'), 0.6266576024101875)]

## 단어들의 조합으로 도서 추천 (기억 속의 책 찾기)
- positive word와 negative word를 입력받아 해당 조합에 따른 유사 단어를 추출한 후, 이 단어들의 벡터 합과 가장 높은 유사도의 도서 내용을 갖는 도서를 추천해주는 함수

In [4]:
def book_search(model_name, dataframe):
    
    # 모델 호출
    try:
        model = Word2Vec.load(model_name)
    except:
        model = FastText.load(model_name)
    
    # 데이터프레임 호출
    book = pd.read_csv(dataframe)
    book_information = zip(book['제목'],book['저자'], book['카테고리'])
    
    k = klt2023()
    
    # positive, negative word 입력
    posit = input('positive_word (여러 단어를 쓸 경우, 띄어쓰기로 구분해주세요.):').split()
    negat = input('negative_word (여러 단어를 쓸 경우, 띄어쓰기로 구분해주세요.):').split()
    
    # 입력단어에 따른 유사 단어 출력
    if (len(posit)>0) and (len(negat)>0):
        word = model.wv.most_similar(positive=posit, negative=negat)
    elif (len(posit)>0):
        word = model.wv.most_similar(positive=posit)
    else:
        word = model.wv.most_similar(negative=negat)
    
    # 유사 단어들의 벡터 합
    w_vector = np.zeros(400)
    for w,c in word:
        w_vector += model.wv.get_vector(w)
    
    # 책벡터와 유사도 계산
    cos_num = []
    for i in range(len(book)):
        book_vector = [np.float(i) for i in book['책소개_벡터'][i][1:-1].split()]
        nn = cosine_similarity(w_vector, book_vector)
        cos_num.append(nn.astype('float'))
    
    # dictionary 생성
    if len(cos_num) == len(book):
        cos_dic = dict(zip(book_information, cos_num))
        book_similar = sorted(cos_dic.items(), key=lambda x: x[1], reverse = True)
        return book_similar[:1]
    else:
         return len(cos_num)

In [18]:
# word2vec은 학습 데이터에 없는 단어 벡터는 만들어 내지 못하는 단점이 있기 때문에
# 해당 예시에서는 '아이'라는 단어가 없어 오류가 난 것을 볼 수 있음
book_search('word2vec_book_wiki.model', 'books_vector_wiki.csv')

positive_word (여러 단어를 쓸 경우, 띄어쓰기로 구분해주세요.):가족 사랑 아이 공부
negative_word (여러 단어를 쓸 경우, 띄어쓰기로 구분해주세요.):전쟁 공주 연애


KeyError: "Key '아이' not present"

In [19]:
book_search('word2vec_book_wiki.model', 'books_vector_wiki.csv')

positive_word (여러 단어를 쓸 경우, 띄어쓰기로 구분해주세요.):모험 전쟁 성공
negative_word (여러 단어를 쓸 경우, 띄어쓰기로 구분해주세요.):공주 연애


[(('이미 시작된 전쟁', '이철', '정치/사회'), 0.27360490246192454)]

In [20]:
book_search('fasttext_book_wiki.model', 'books_vector_wiki.csv')

positive_word (여러 단어를 쓸 경우, 띄어쓰기로 구분해주세요.):모험 전쟁 성공
negative_word (여러 단어를 쓸 경우, 띄어쓰기로 구분해주세요.):공주 연애


[(('로마인 이야기 2: 한니발 전쟁', '시오노 나나미', '역사/문화'), 0.5823006581838892)]

In [10]:
book_search('word2vec_book_wiki.model', 'books_vector_wiki.csv')

positive_word (여러 단어를 쓸 경우, 띄어쓰기로 구분해주세요.):마법 성 판타지
negative_word (여러 단어를 쓸 경우, 띄어쓰기로 구분해주세요.):


[(('해리 포터 시리즈 1~4권 세트(해리포터 20주년 개정판)', 'J. K. 롤링', '소설'), 0.3743759220949477)]

In [11]:
book_search('fasttext_book_wiki.model', 'books_vector_wiki.csv')

positive_word (여러 단어를 쓸 경우, 띄어쓰기로 구분해주세요.):마법 성 판타지
negative_word (여러 단어를 쓸 경우, 띄어쓰기로 구분해주세요.):


[(('해리 포터 시리즈 1~4권 세트(해리포터 20주년 개정판)', 'J. K. 롤링', '소설'), 0.6568774500743274)]

## 입력한 단어와 가장 유사한 카테고리의 도서 추천

In [32]:
def bestseller(model_name, dataframe, word1, n):
    
    # 모델 호출
    try:
        model = Word2Vec.load(model_name)
    except:
        model = FastText.load(model_name)
    
    # 데이터프레임 호출
    book = pd.read_csv(dataframe)
    
    k = klt2023()
    
    # 입력 단어와 카테고리 유사도 계산
    categ=[i.split('/') for i in book['카테고리']]
    category = list(set(itertools.chain(*categ)))
    simil_num=0
    for i in category:
        simil = model.wv.similarity(u'{}'.format(word1), u'{}'.format(i))
        if simil > simil_num:
            simil_num = simil
            ct = i
    
    # 해당 카테고리에 해당하는 book 목록으로 재정의
    book = book[book['카테고리'].apply(lambda x : ct in x)].reset_index(drop=True)
    book_information = zip(book['제목'],book['저자'])
    
    # 책소개 벡터와 유사도 계산
    cos_num = []
    word_vec = model.wv.get_vector(word1)
    for i in range(len(book)):
        book_vector = [np.float(i) for i in book['책소개_벡터'][i][1:-1].split()]
        nn = cosine_similarity(word_vec, book_vector)
        cos_num.append(nn.astype('float'))
    
    # dictionary 생성
    if len(cos_num) == len(book):
        cos_dic = dict(zip(book_information, cos_num))
        book_similar = sorted(cos_dic.items(), key=lambda x: x[1], reverse = True)
        return simil, ct, book_similar[:n]
    else:
        return len(cos_num)

In [36]:
bestseller('fasttext_book_wiki.model', 'books_vector_wiki.csv','성공',5)

(0.04292066,
 '대중문화',
 [(('SAVE THE CAT!: 흥행하는 영화 시나리오의 8가지 법칙', '블레이크 스나이더'), 0.44993079561155186),
  (('우리는 왜 임영웅을 사랑하는가', '조위'), 0.34890712733005674),
  (('시나리오 어떻게 쓸 것인가 세트', '로버트 맥키'), 0.343034047968187),
  (('데이비드 호크니', '마르코 리빙스턴'), 0.327195837798465),
  (('연기하지 않는 연기', '해럴드 거스킨'), 0.31482199930558946)])

In [38]:
bestseller('fasttext_book_wiki.model', 'books_vector_wiki.csv','모험',5)

(0.1325179,
 '소설',
 [(('해리 포터 시리즈 1~4권 세트(해리포터 20주년 개정판)', 'J. K. 롤링'), 0.6852860696465063),
  (('해리 포터와 마법사의 돌 1(해리포터 20주년 개정판)', 'J. K. 롤링'), 0.6755744803449041),
  (('해리 포터와 마법사의 돌 2(해리포터 20주년 개정판)', 'J. K. 롤링'), 0.6755744803449041),
  (('해리 포터와 마법사의 돌(해리포터 20주년 개정판)', 'J. K. 롤링'), 0.6755744803449041),
  (('해리 포터와 아즈카반의 죄수 1(해리포터 20주년 개정판)', 'J. K. 롤링'), 0.665523252850332)])